Importing basic packages

In [1]:
import numpy as np
import pandas as pd
import warnings

Data loading and preprocessing

In [2]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [ ]:
groceries = pd.read_csv('Groceries_datasett.csv')
groceries
groc = groceries

In [ ]:
groc.shape
groc.info()

In [ ]:
groc['itemDescription']=groc['itemDescription'].str.strip()
groc

In [ ]:
groc['Date']=pd.to_datetime(groc['Date'])
groc['day_of_week'] = groc['Date'].dt.dayofweek
groc['day_of_week_name'] = groc['Date'].dt.day_name()
groc

In [ ]:
# This line should be applied to the DataFrame which needs the 'Quantity' column.
# If it's 'groc', then it should be groc['Quantity'] = 1
groc['Quantity'] = 1

# Group by 'day_of_week_name' and sum the 'Quantity' column
df_tot = groc.groupby(['day_of_week_name'])["Quantity"].sum()

# Convert the resulting Series to a DataFrame
df_total = pd.DataFrame(df_tot).reset_index().sort_values(by='Quantity', ascending=False)

# Display the first few rows
df_total


In [ ]:
#top 25 products sold
import matplotlib.pyplot as plt
#import seaborn as sns
import plotly.express as px
top_25=groc.itemDescription.value_counts().sort_values(ascending=False)[0:25]
fig = px.bar(top_25,color=top_25.index, labels={'value':'Quantity Sold','index':'GroceryItems'})
fig.update_layout(showlegend=False, title_text='Top 25 Groceries Sold',title_x=0.5, title={'font':{'size':20}})
fig.show()

In [ ]:
#bottom 25 products sold
bot_25=groc.itemDescription.value_counts().sort_values(ascending=False)[-25:]
fig = px.bar(bot_25,color=bot_25.index, labels={'value':'Quantity Sold','index':'GroceryItems'})
fig.update_layout(showlegend=False, title_text='Bottom 25 Groceries Sold',title_x=0.5, title={'font':{'size':20}})
fig.show()

In [ ]:
#frequency of items sold
item_freq = groc.groupby(pd.Grouper(key='itemDescription')).size().reset_index(name='count')
fig = px.treemap(item_freq, path=['itemDescription'], values='count')
fig.update_layout(title_text='Frequency of the Groceries used', title_x=0.5, title_font=dict(size=18))
fig.update_traces(textinfo="label+value")
fig.show()

In [ ]:
#top 25 customers visiting across the dataset
top_25c = groc.groupby('Member_number').agg(PurchaseQuantity=('itemDescription','count')).sort_values(by='PurchaseQuantity',ascending=False)[0:25]
top_25c.plot(kind='bar', figsize=(15,7), legend=None)
plt.title('Top 25 customers', fontsize=20)
plt.xlabel('Customer Number', fontsize=15)
plt.ylabel('Purchase Quantity', fontsize=15)
plt.show()

In [ ]:
# Implementation of FP Algorithm
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load your dataset
# For this example, let's assume 'groceries_dataset' is a list of lists,
# where each inner list is a transaction.
# Example: groceries_dataset = [['milk', 'bread'], ['bread', 'butter'], ...]

# Convert dataset to the right format
te = TransactionEncoder()
te_ary = te.fit(groc).transform(groc)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Apply the FP-Growth algorithm
frequent_itemsets = fpgrowth(df, min_support=0.05, use_colnames=True)

# Generate Association Rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# Display the rules
print(rules)

In [ ]:
baskets=groc.groupby(['Member_number','itemDescription'])['itemDescription'].count().unstack()
baskets

In [ ]:
#checking for not null values
baskets.notnull().sum()

In [ ]:
baskets=baskets.fillna(0).reset_index()
baskets.head()

In [ ]:
def convert_values(value):
    if value <= 0:
        return 0
    elif value >=1:
        return 1

In [ ]:
baskets = baskets.iloc[:, 1:baskets.shape[1]].applymap(convert_values)
baskets.head()

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

In [ ]:
df_new = pd.DataFrame(baskets)

In [ ]:
#the itemsets for which the support is greater or equal to 0.05
freq_items = apriori(df_new, min_support=0.05, use_colnames=True, max_len=3).sort_values(by='support')
freq_items.head(10)

In [ ]:
rules=association_rules(freq_items, metric="lift", min_threshold=1).sort_values('lift',ascending=False)
rules=rules[['antecedents','consequents','support','confidence','lift']]
rules.head()

In [ ]:
import seaborn as sns
#convert antecedents and consequents into strings
rules['antecedents']=rules['antecedents'].apply(lambda a: ','.join(list(a)))
rules['consequents']=rules['consequents'].apply(lambda a: ','.join(list(a)))
print(rules[['antecedents','consequents']])

In [ ]:
groc['Month'] = groc.Date.dt.month
groc['Month'] = groc['Date'].apply(lambda x:x.month)

In [ ]:
#Season : 1=fall 2=Summer 3=Winter
#mapping months to seasons
month_to_season = {1: 3, 2: 3, 3: 3, 4: 2, 5: 2, 6: 2, 7: 2, 8: 1, 9: 1, 10: 1, 11: 3, 12: 3}
groc['season']=groc['Month'].map(month_to_season)
groc

In [ ]:
def seasonwise(x):
    groc_n=groc[groc['itemDescription']==x].groupby('season').count()
    print(groc_n)
seasonwise('mango')

In [ ]:
#number of each item bought in total
groc_quantity = groc.groupby(['itemDescription','season'])['Quantity'].count()
groc_quantity = groc_quantity.reset_index()
groc_quantity = groc_quantity.sort_values(by=['Quantity','itemDescription'],ascending=False)
groc_quantity= groc_quantity.reset_index()
groc_quantity

In [ ]:
groc_winter=groc_quantity[groc_quantity['season']==3]
groc_winter

In [ ]:
grouped_data = groc_winter.groupby('itemDescription')['Quantity'].sum().reset_index()

# Sort the grouped data by 'Quantity' in descending order and take the top 25
top_25_winter = grouped_data.sort_values(by='Quantity', ascending=False).head(25)

plt.figure(figsize=(15, 7))
plt.bar(top_25_winter['itemDescription'], top_25_winter['Quantity'])
plt.xlabel('GroceriesItems')
plt.ylabel('QuantitySold')
plt.title('Top 25 Groceries Sold in Winter')
plt.xticks(rotation=90)  # Rotate labels by 90 degrees

# Show the graph
plt.show()

In [ ]:
groc_winter=groc[groc['season']==3]
# Group by 'day_of_week_name' and sum the 'Quantity' column
df_tot = groc_winter.groupby(['day_of_week_name'])['Quantity'].sum()

# Convert the resulting Series to a DataFrame
df_total = pd.DataFrame(df_tot).reset_index()
df_total=df_total.sort_values(by='Quantity', ascending=False)
df_total

In [ ]:
baskets=groc_winter.groupby(['Member_number','itemDescription'])['itemDescription'].count().unstack()
baskets

In [ ]:
#checking for not null values
baskets.notnull().sum()
baskets=baskets.fillna(0).reset_index()
baskets.head()

In [ ]:
def convert_values(value):
    if value <= 0:
        return 0
    elif value >=1:
        return 1
baskets = baskets.iloc[:, 1:baskets.shape[1]].applymap(convert_values)
baskets.head()

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
df_new = pd.DataFrame(baskets)
freq_items = apriori(df_new, min_support=0.05, use_colnames=True, max_len=3).sort_values(by='support')
freq_items.head(10)

In [ ]:
rules=association_rules(freq_items, metric="lift", min_threshold=1).sort_values('support',ascending=False)
rules=rules[['antecedents','consequents','support','confidence','lift']]
rules.head()

In [ ]:
#creating association rules for indicating antecedents and consequent items
rules=association_rules(freq_items, metric="lift", min_threshold=1).sort_values('lift',ascending=False)
rules=rules[['antecedents','consequents','support','confidence','lift']]
rules.head()
import seaborn as sns
#convert antecedents and consequents into strings
rules['antecedents']=rules['antecedents'].apply(lambda a: ','.join(list(a)))
rules['consequents']=rules['consequents'].apply(lambda a: ','.join(list(a)))
print(rules[['antecedents','consequents']])

In [ ]:
groc_fall=groc_quantity[groc_quantity['season']==1]
groc_fall

In [ ]:
grouped_data = groc_fall.groupby('itemDescription')['Quantity'].sum().reset_index()

# Sort the grouped data by 'Quantity' in descending order and take the top 25
top_25_fall = grouped_data.sort_values(by='Quantity', ascending=False).head(25)

plt.figure(figsize=(15, 7))
plt.bar(top_25_fall['itemDescription'], top_25_fall['Quantity'])
plt.xlabel('GroceriesItems')
plt.ylabel('QuantitySold')
plt.title('Top 25 Groceries Sold in Fall')
plt.xticks(rotation=90)  # Rotate labels by 90 degrees

# Show the graph
plt.show()

In [ ]:
groc_fall=groc[groc['season']==1]
# Group by 'day_of_week_name' and sum the 'Quantity' column
df_tot = groc_fall.groupby(['day_of_week_name'])['Quantity'].sum()

# Convert the resulting Series to a DataFrame
df_total = pd.DataFrame(df_tot).reset_index()
df_total=df_total.sort_values(by='Quantity', ascending= False)
df_total

In [ ]:
baskets=groc_fall.groupby(['Member_number','itemDescription'])['itemDescription'].count().unstack()
baskets

In [ ]:
#checking for not null values
baskets.notnull().sum()
baskets=baskets.fillna(0).reset_index()
baskets.head()

In [ ]:
def convert_values(value):
    if value <= 0:
        return 0
    elif value >=1:
        return 1
baskets = baskets.iloc[:, 1:baskets.shape[1]].applymap(convert_values)
baskets.head()

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
df_new = pd.DataFrame(baskets)
freq_items = apriori(df_new, min_support=0.03, use_colnames=True, max_len=3).sort_values(by='support')
freq_items.head(10)

In [ ]:
rules=association_rules(freq_items, metric="lift", min_threshold=1).sort_values('support',ascending=False)
rules=rules[['antecedents','consequents','support','confidence','lift']]
rules.head()

In [ ]:
groc_summer=groc_quantity[groc_quantity['season']==2]
groc_summer

In [ ]:
grouped_data = groc_summer.groupby('itemDescription')['Quantity'].sum().reset_index()

# Sort the grouped data by 'Quantity' in descending order and take the top 25
top_25_summer = grouped_data.sort_values(by='Quantity', ascending=False).head(25)

plt.figure(figsize=(15, 7))
plt.bar(top_25_summer['itemDescription'], top_25_summer['Quantity'])
plt.xlabel('GroceriesItems')
plt.ylabel('QuantitySold')
plt.title('Top 25 Groceries Sold in Summer')
plt.xticks(rotation=90)  # Rotate labels by 90 degrees

# Show the graph
plt.show()

In [ ]:
groc_summer=groc[groc['season']==2]
# Group by 'day_of_week_name' and sum the 'Quantity' column
df_tot = groc_summer.groupby(['day_of_week_name'])['Quantity'].sum()

# Convert the resulting Series to a DataFrame
df_total = pd.DataFrame(df_tot).reset_index()
df_total=df_total.sort_values(by='Quantity', ascending=False)
df_total

In [ ]:
baskets=groc_summer.groupby(['Member_number','itemDescription'])['itemDescription'].count().unstack()
baskets

In [ ]:
#checking for not null values
baskets.notnull().sum()
baskets=baskets.fillna(0).reset_index()
baskets.head()

In [ ]:
def convert_values(value):
    if value <= 0:
        return 0
    elif value >=1:
        return 1
baskets = baskets.iloc[:, 1:baskets.shape[1]].applymap(convert_values)
baskets.head()

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
df_new = pd.DataFrame(baskets)
freq_items = apriori(df_new, min_support=0.05, use_colnames=True, max_len=3).sort_values(by='support')
freq_items.head(10)

In [ ]:
rules=association_rules(freq_items, metric="lift", min_threshold=1).sort_values('support',ascending=False)
rules=rules[['antecedents','consequents','support','confidence','lift']]
rules.head()